In [14]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.constants import physical_constants
from scipy.sparse import diags
from scipy.sparse.linalg import eigsh
import pandas as pd

# Task 1: Numerically solving for eigenstates and eigenvalues of an arbitrary 1D potential

Description: Obtain the energy eigenvalues $E_n$ and wavefunctions $\psi_n(r)$ for the anharmonic Morse potential (below). Note that value of the parameters correspond the hydrogen fluoride. Tabulate $E_n$ for $n=0$ to $5$, and plot the corresponding $\psi_n(r)$.

$$V = D_e [1-e^{-\alpha x}]^2$$
- **Equilibrium bond energy**:
    $$D=6.091\times10^{-19} \text{ J}$$
- **Equilibrium bond length**:
    $$r_0=9.109\times10^{-11} \text{ m}, \quad x=r-r_0$$
- **Force constant**:
    $$k=1.039\times10^{3} \ \text{J}\text{m}^{-2}, \quad \alpha=\sqrt{ k / 2D_e}$$

## Calculate $\alpha$ using the given $k$ and $D_e$

In [2]:
import numpy as np

k = 1.039e3         # J/m^2
D_e = 6.091e-19     # J

alpha = np.sqrt(k / (2 * D_e))  # 1/m
print(f"Alpha = {alpha:.3e} 1/m")

Alpha = 2.920e+10 1/m


## Reduced mass, $\mu$ of H-F molecule

$$\mu = \frac{m_\text{H} m_\text{F}}{m_\text{H} + m_\text{F}}

In [4]:
m_H_u = 1.00784     # atomic mass units
m_F_u = 18.998403   # atomic mass units
u = 1.660539e-27    # kg

m_H = m_H_u * u
m_F = m_F_u * u
mu = (m_H * m_F) / (m_H + m_F)
print(f"Reduced mass: {mu:.3e} kg")

Reduced mass: 1.589e-27 kg


## Set up spatial grid

In [6]:
xmin = -5e-10   # m
xmax = 5e-10    # m
N = 1000        # number of grid points
x = np.linspace(xmin, xmax, N)
dx = x[1] - x[0]

## Calculate the potential $V(x)$ at each point on the grid

In [7]:
V = D_e * (1 - np.exp(-alpha * x))**2

## Set up the kinetic energy operator ⚠️

In [8]:
hbar = 1.0545718e-34    # Js

# Construct the second derivative operator (kinetic energy term)
T_coeff = (hbar**2) / (2 * mu * dx**2)
diagonal = -2 * np.ones(N) * T_coeff
off_diagonal = np.ones(N - 1) * T_coeff

# Assemble the kinetic energy matrix
from scipy.sparse import diags

T = diags([off_diagonal, diagonal, off_diagonal], offsets=[-1,0,1])

<DIAgonal sparse matrix of dtype 'float64'
	with 2998 stored elements (3 diagonals) and shape (1000, 1000)>
  Coords	Values
  (1, 0)	3.4918940745861605e-18
  (2, 1)	3.4918940745861605e-18
  (3, 2)	3.4918940745861605e-18
  (4, 3)	3.4918940745861605e-18
  (5, 4)	3.4918940745861605e-18
  (6, 5)	3.4918940745861605e-18
  (7, 6)	3.4918940745861605e-18
  (8, 7)	3.4918940745861605e-18
  (9, 8)	3.4918940745861605e-18
  (10, 9)	3.4918940745861605e-18
  (11, 10)	3.4918940745861605e-18
  (12, 11)	3.4918940745861605e-18
  (13, 12)	3.4918940745861605e-18
  (14, 13)	3.4918940745861605e-18
  (15, 14)	3.4918940745861605e-18
  (16, 15)	3.4918940745861605e-18
  (17, 16)	3.4918940745861605e-18
  (18, 17)	3.4918940745861605e-18
  (19, 18)	3.4918940745861605e-18
  (20, 19)	3.4918940745861605e-18
  (21, 20)	3.4918940745861605e-18
  (22, 21)	3.4918940745861605e-18
  (23, 22)	3.4918940745861605e-18
  (24, 23)	3.4918940745861605e-18
  (25, 24)	3.4918940745861605e-18
  :	:
  (974, 975)	3.4918940745861605e-18
  (

## Construct the Hamiltonian Matrix
Combine the kinetic and potential energy terms

In [9]:
from scipy.sparse import dia_matrix

# Potential energy matrix (diagonal matrix ⚠️
V_matrix = dia_matrix((V, [0]), shape=(N,N))

# Hamiltonian matrix
H = T + V_matrix

## Solving the Schrodinger Equation ⚠️

In [11]:
from scipy.sparse.linalg import eigsh

# Number of eigenvalues and eigenvectors to compute
num_eigenvalues = 6

# Compute the lowest eigenvalues and corresponding eigenvectors
eigenvalues, eigenvectors = eigsh(H, k=num_eigenvalues, which='SM')

ArpackNoConvergence: ARPACK error -1: No convergence (10001 iterations, 0/6 eigenvectors converged)

## Converting Eigenvalues to Physical Units

In [12]:
# Eigenvalues are already in Joules
E_n = eigenvalues   # J

NameError: name 'eigenvalues' is not defined

## Normalizing the Eigenfunctions ⚠️
Normalize the eigenfunctions so that the integral over all space is 1

In [ ]:
# Normalize eigenfunctions
psi_n = []
for i in range(num_eigenvalues):
    psi = eigenvectors[:, i]
    
    # Normalize
    norm = np.sqrt(np.sum(np.abs(psi)**2) + dx)    
    psi_n.append(psi / norm) 

## Tabulate the Energy Eigenvalues

In [ ]:
import pandas as pd 

data = {'n': np.arange(num_eigenvalues), 'E_n (J)': E_n}
df = pd.DataFrame(data)
print(df)

## Plot the Wavefunctions

In [13]:
import matplotlib.pyplot as plt 

plt.figure(figsize=(10, 6))
for i in range(num_eigenvalues):
    plt.plot(x + r_0, psi_n[i] + E_n[i], label=f'n={i}')
    
# Plot the potential for reference
plt.plot(x + r_0, V, 'k-', label='Potential V(x)')

plt.xlabel('Position r (m)')
plt.ylabel('Energy (J)')
plt.title('Morse Potential Wavefunction')
plt.legend()
plt.show()

NameError: name 'r_0' is not defined

<Figure size 1000x600 with 0 Axes>